In [1]:
import requests
import pandas

Playing around with the USCIS immigration forms data gave this URL for a search result, which has PDF and CSVs that go back to 2012.

In [2]:
r = requests.get('https://www.uscis.gov/tools/reports-studies/immigration-forms-data?topic_id=20709&field_native_doc_issue_date_value%5Bvalue%5D%5Bmonth%5D=&field_native_doc_issue_date_value_1%5Bvalue%5D%5Byear%5D=&combined=&items_per_page=200')

In [3]:
r.status_code

200

Looks like the request was successful, let's get links to the data sources.

In [4]:
from bs4 import BeautifulSoup

BeautifulSoup is a library for HTML parsing, super useful for web scraping.

In [5]:
soup = BeautifulSoup(r.content)

Inspecting one of the links for a CSV, I see that it has these properties that I can query by.

In [6]:
csvs = soup.findAll("a", attrs={"type":"text/csv"})
len(csvs), csvs[0]

(29,
 <a href="https://www.uscis.gov/sites/default/files/USCIS/Resources/Reports%20and%20Studies/Immigration%20Forms%20Data/Naturalization%20Data/N400_performancedata_fy2019_qtr2.csv" length="20453" type="text/csv"> Form N-400, Application for Naturalization, by Category of Naturalization, Case Status, and USCIS Field Office Location (Fiscal Year 2019, 2nd Quarter, Jan. 1-Mar. 31, 2019)  (CSV, 20 KB)</a>)

Looks like we should cut off the first 4 lines. We can iterate through all the HTML objects and query on the link it's linking to, saving to files. And here's how we can name them.

In [7]:
csvs[0].text.split('(')[1].split(')')[0]

'Fiscal Year 2019, 2nd Quarter, Jan. 1-Mar. 31, 2019'

In [8]:
# another quirk of the data is it's encoded in a less-commonly used encoding, discovered what it is by trying a few
encoding="windows-1252"

In [9]:
dfs = []
for obj in csvs:
    text = requests.get(obj.get('href')).content.decode(encoding)
    csv_fname = obj.text.split('(')[1].split(')')[0]
    csv_fname = csv_fname.replace("Fiscal Year", "FY")
    csv_fname = csv_fname.replace("Quarter", "Q")
    with open('data/USCIS/' + csv_fname + '.csv', 'w') as f:
        f.write(text)

In [10]:
import os
os.listdir('data/USCIS')

['FY 2013, 4th Q, July 1- Sept. 30, 2013.csv',
 'FY 2014, 1st Q, Oct. 1-Dec. 31, 2013.csv',
 'FY 2014, 2nd Q, Jan. 1-March 31, 2014.csv',
 'FY 2014, 3rd Q, April 1-June 30, 2014.csv',
 'FY 2014, 4th Q, July 1-Sept. 30, 2014.csv',
 'FY 2015, 1st Q, Oct. 1-Dec. 31, 2014.csv',
 'FY 2015, 2nd Q, Jan. 1-March 31, 2015.csv',
 'FY 2015, 3rd Q, April 1-June 30, 2015.csv',
 'FY 2015, 4th Q, July 1-Sept. 30, 2015.csv',
 'FY 2016, 1st Q, Oct. 1-Dec.31, 2015.csv',
 'FY 2016, 2nd Q, Jan. 1-March 31, 2016.csv',
 'FY 2016, 3rd Q, April 1-June 30, 2016.csv',
 'FY 2016, 4th Q, July 1-Sept. 30, 2016.csv',
 'FY 2017, 1st Q, Oct. 1-Dec. 31, 2016.csv',
 'FY 2017, 2nd Q, Jan. 1-March 31, 2017.csv',
 'FY 2017, 3rd Q, April 1-June 30, 2017.csv',
 'FY 2017, 4th Q, July 1-Sept. 30, 2017.csv',
 'FY 2018, 1st Q, Oct. 1-Dec. 31, 2017.csv',
 'FY 2018, 2nd Q, Jan. 1-March 31, 2018.csv',
 'FY 2018, 3rd Q, April 1-June 30, 2018.csv',
 'FY 2018, 4th Q, July 1-Sept. 30, 2018.csv',
 'FY 2019, 1st Q, Oct. 1-Dec. 31, 2018.

Looks like the format is different for some of them, let's take a peek at the first 10 lines of each file.

In [11]:
office_name = 'Anchorage'
for fname in os.listdir('data/USCIS'):
    print(fname)
    with open('data/USCIS/'+fname) as f:
        text = f.read().split('\n')
        for i,line in enumerate(text):
            if office_name in line:
                print('\t',line)
                items = [item.strip() for item in line.split(',')]
                print('\trow', i, 'col', items.index(office_name)+1)

FY 2013, 4th Q, July 1- Sept. 30, 2013.csv
	 ,Anchorage , 257 , 316 , 39 , 365 , D , 15 , D , 20 , 263 , 331 , 42 , 385 ,
	row 18 col 2
FY 2014, 1st Q, Oct. 1-Dec. 31, 2013.csv
	 ,Anchorage ,274,174,30,447,10,18, -   ,18,284,192,30,465
	row 18 col 2
FY 2014, 2nd Q, Jan. 1-March 31, 2014.csv
	 ,Anchorage ,297,210,48,533, D , D , D ,22,305,219,50,555
	row 20 col 2
FY 2014, 3rd Q, April 1-June 30, 2014.csv
	 ,Anchorage ,310,270,48,525, D ,16, -   ,10,313,286,48,535
	row 20 col 2
FY 2014, 4th Q, July 1-Sept. 30, 2014.csv
	 ,Anchorage ,272,370,36,404, D ,13, -   ,11,278,383,36,415
	row 16 col 2
FY 2015, 1st Q, Oct. 1-Dec. 31, 2014.csv
	 ,Anchorage ,311,215,35,501, D , D , D ,20,320,217,36,521
	row 16 col 2
FY 2015, 2nd Q, Jan. 1-March 31, 2015.csv
	 ,Anchorage ,ANC,312,259,37,539,6,6,1,15,318,265,38,554
	row 16 col 2
FY 2015, 3rd Q, April 1-June 30, 2015.csv
	 ,Anchorage ,ANC,300,198,27,603,9,5, -   ,20,309,203,27,623
	row 16 col 2
FY 2015, 4th Q, July 1-Sept. 30, 2015.csv
	 ,Anchorage ,ANC

Looks like it's always the 2nd column, so that's good. Let's try loading one into a CSV and dropping all the rows without field office names. Pandas columns are 0-indexed so the second column will be called the 1st.

In [13]:
df = pandas.read_csv('data/USCIS/FY 2019, 2nd Q, Jan. 1-Mar. 31, 2019.csv', encoding=encoding)
df['Unnamed: 1'].unique()

array([nan, ' Montgomery ', ' Anchorage  ', ' Phoenix  ', ' Tucson ',
       ' Fort Smith  ', ' Chula Vista ', ' Fresno  ', ' Imperial ',
       ' Los Angeles  ', ' Los Angeles County  ', ' Sacramento  ',
       ' San Bernardino ', ' San Diego  ', ' San Fernando Valley ',
       ' San Francisco ', ' San Jose  ', ' Santa Ana ', ' Denver  ',
       ' Hartford  ', ' Dover AFB ', ' Washington  ', ' Fort Meyers ',
       ' Hialeah ', ' Jacksonville ', ' Kendall ', ' Miami  ',
       ' Oakland Park ', ' Orlando  ', ' Tampa  ', ' West Palm Beach ',
       ' Atlanta  ', ' Honolulu  ', ' Boise  ', ' Chicago  ',
       ' Indianapolis  ', ' Des Moines ', ' Wichita ', ' Louisville  ',
       ' New Orleans  ', ' Portland  ', ' Baltimore  ', ' Boston  ',
       ' Lawrence ', ' Detroit  ', ' St. Paul  ', ' Kansas City  ',
       ' St. Louis  ', ' Helena  ', ' Omaha  ', ' Las Vegas  ', ' Reno  ',
       ' Manchester  ', ' Mount Laurel ', ' Newark  ', ' Albuquerque  ',
       ' Albany  ', ' Brooklyn ',

In [14]:
df.dropna(subset=['Unnamed: 1'])

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
11,NaN,Montgomery,MGA,"1,009","1,675",122,"3,465",40,D,D,74,"1,049","1,681",123,"3,539"
13,NaN,Anchorage,ANC,301,162,38,687,D,10,D,23,304,172,39,710
15,NaN,Phoenix,PHO,"2,461","2,188",373,"12,620",D,D,D,30,"2,464","2,193",375,"12,650"
16,NaN,Tucson,TUC,584,756,80,"1,224",-,D,-,D,584,764,80,"1,229"
18,NaN,Fort Smith,FSA,370,278,14,"1,193",-,D,D,D,370,280,15,"1,196"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,NaN,Moscow,NaN,N/A,N/A,N/A,N/A,-,-,-,-,-,-,-,-
218,NaN,Johannesburg,NaN,N/A,N/A,N/A,N/A,-,-,-,-,-,-,-,-
220,NaN,Seoul,NaN,N/A,N/A,N/A,N/A,185,107,-,84,184,108,-,82
222,NaN,Bangkok,NaN,N/A,N/A,N/A,N/A,-,D,-,D,-,D,-,D


I spend a bunch of time trying to automatically extract the correct column names before realizing I could save time by just deleting everything above what I wanted to use.

In [167]:
dfs = []

for fname in os.listdir('data/USCIS/massaged'):
    if 'FYs' in fname:
        # skip the 2010-2013 CSVs for now
        continue
    df = pandas.read_csv('data/USCIS/massaged/' + fname, header=0, encoding=encoding)
    # make sure whitespace is remove so all column names are exactly the same
    df.columns = [name.strip() for name in df.columns]
    # rename a column, delete any rows that are blank in that column, and set it as the index
    df = df.rename(columns={'Unnamed: 1':"field_office"}).dropna(subset=['field_office'])
    # remove the whitespace from the names
    df['field_office'] = df.field_office.map(lambda name: name.strip())
    # keep track of which quarter this is from
    df['quarter'] = fname.split('.csv')[0]
    dfs.append(df)
    # keep track of how many columns are in each quarter's CSV and whether there are any non-unique column names (looks like there aren't)
    print(fname, len(df.columns), len(set(df.columns)))

FY 2013, 4th Q, July 1- Sept. 30, 2013.csv 15 15
FY 2014, 1st Q, Oct. 1-Dec. 31, 2013.csv 15 15
FY 2014, 2nd Q, Jan. 1-March 31, 2014.csv 15 15
FY 2014, 3rd Q, April 1-June 30, 2014.csv 15 15
FY 2014, 4th Q, July 1-Sept. 30, 2014.csv 15 15
FY 2015, 1st Q, Oct. 1-Dec. 31, 2014.csv 15 15
FY 2015, 2nd Q, Jan. 1-March 31, 2015.csv 16 16
FY 2015, 3rd Q, April 1-June 30, 2015.csv 16 16
FY 2015, 4th Q, July 1-Sept. 30, 2015.csv 16 16
FY 2016, 1st Q, Oct. 1-Dec.31, 2015.csv 16 16
FY 2016, 2nd Q, Jan. 1-March 31, 2016.csv 16 16
FY 2016, 3rd Q, April 1-June 30, 2016.csv 16 16
FY 2016, 4th Q, July 1-Sept. 30, 2016.csv 16 16
FY 2017, 1st Q, Oct. 1-Dec. 31, 2016.csv 16 16
FY 2017, 2nd Q, Jan. 1-March 31, 2017.csv 16 16
FY 2017, 3rd Q, April 1-June 30, 2017.csv 16 16
FY 2017, 4th Q, July 1-Sept. 30, 2017.csv 16 16
FY 2018, 1st Q, Oct. 1-Dec. 31, 2017.csv 16 16
FY 2018, 2nd Q, Jan. 1-March 31, 2018.csv 16 16
FY 2018, 3rd Q, April 1-June 30, 2018.csv 16 16
FY 2018, 4th Q, July 1-Sept. 30, 2018.csv 16 

In [168]:
df = pandas.concat(dfs)
df

,Unnamed: 0,field_office,Applications Received2,Approved3,Denied4,Pending5,Applications Received2.1,Approved3.1,Denied4.1,Pending5.1,...,USCIS Field Office or Service Center Location,Unnamed: 2,Applications Received2 .1,Approved3 .1,Denied4 .1,Pending5 .1,Applications Received2 .2,Approved3 .2,Denied4 .2,Pending5 .2
11,NaN,Anchorage,257,316,39,365,D,15,D,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,Phoenix,"2,759","2,399",312,"4,700",16,12,D,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,Tucson,636,513,80,"1,166",10,D,D,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,Fort Smith,232,202,22,411,D,D,-,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,Chula Vista,"1,582","1,928",121,"2,004",36,50,D,108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,NaN,Moscow,N/A,N/A,N/A,N/A,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423,NaN,Johannesburg,N/A,N/A,N/A,N/A,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,NaN,Seoul,N/A,N/A,N/A,N/A,185,107,-,84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,NaN,Bangkok,N/A,N/A,N/A,N/A,-,D,-,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There it is! Let's check out just the non-military accepted columns, by quarter

In [169]:
df[['Approved3', 'quarter']]

,Approved3,quarter
11,316,"FY 2013, 4th Q, July 1- Sept. 30, 2013"
15,"2,399","FY 2013, 4th Q, July 1- Sept. 30, 2013"
17,513,"FY 2013, 4th Q, July 1- Sept. 30, 2013"
21,202,"FY 2013, 4th Q, July 1- Sept. 30, 2013"
25,"1,928","FY 2013, 4th Q, July 1- Sept. 30, 2013"
...,...,...
419,N/A,"FY 2019, 2nd Q, Jan. 1-Mar. 31, 2019"
423,N/A,"FY 2019, 2nd Q, Jan. 1-Mar. 31, 2019"
427,N/A,"FY 2019, 2nd Q, Jan. 1-Mar. 31, 2019"
431,N/A,"FY 2019, 2nd Q, Jan. 1-Mar. 31, 2019"


And if we want to narrow down to field office

In [170]:
df[df.field_office=='Anchorage'][['Approved3', 'quarter']]

,Approved3,quarter
11,316,"FY 2013, 4th Q, July 1- Sept. 30, 2013"
11,174,"FY 2014, 1st Q, Oct. 1-Dec. 31, 2013"
11,210,"FY 2014, 2nd Q, Jan. 1-March 31, 2014"
11,270,"FY 2014, 3rd Q, April 1-June 30, 2014"
11,370,"FY 2014, 4th Q, July 1-Sept. 30, 2014"
11,215,"FY 2015, 1st Q, Oct. 1-Dec. 31, 2014"
11,259,"FY 2015, 2nd Q, Jan. 1-March 31, 2015"
11,198,"FY 2015, 3rd Q, April 1-June 30, 2015"
11,269,"FY 2015, 4th Q, July 1-Sept. 30, 2015"
11,202,"FY 2016, 1st Q, Oct. 1-Dec.31, 2015"


In [171]:
df.to_csv('data/master_df.csv')